In [1]:
! pip install pydub

In [3]:
import numpy as np
import pandas as pd
from pydub import AudioSegment
from pydub.generators import Sine
import torch
import colorsys

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [6]:
df = pd.read_csv('/content/galaxy_image_pixel_data.csv')  # Replace with your CSV file path

# Take the first 1000 rows
df_subset = df.head(1000)

In [7]:
# Create an empty audio segment
audio = AudioSegment.silent(duration=0)

### Function to convert RGB to HSV

In [8]:
def rgb_to_hsv(r, g, b):
    return colorsys.rgb_to_hsv(r / 255.0, g / 255.0, b / 255.0)

### Function to map hue to frequency

In [9]:
def hue_to_frequency(hue):
    # Map hue (0-1) to frequency (220-880 Hz)
    return 220 + hue * 660

### Function to map saturation to amplitude

In [10]:
def saturation_to_amplitude(saturation):
    # Map saturation (0-1) to amplitude (0-1)
    return saturation

### Function to map value to duration

In [11]:
def value_to_duration(value):
    # Map value (0-1) to duration (100-500 ms)
    return 100 + value * 400

In [12]:
def hsv_to_sound(hue, saturation, value):
    frequency = hue_to_frequency(hue)
    amplitude = saturation_to_amplitude(saturation)
    duration = value_to_duration(value)
    # Generate sine wave with the calculated frequency and amplitude
    tone = Sine(frequency).to_audio_segment(duration=duration).apply_gain(amplitude * 20 - 20)
    return tone

## Generate sound for each pixel in the subset

In [13]:
for index, row in df_subset.iterrows():
    r, g, b = row['R'], row['G'], row['B']
    hue, saturation, value = rgb_to_hsv(r, g, b)
    tone = hsv_to_sound(hue, saturation, value)
    audio += tone

## Export the final sound file

In [14]:
output_filename = "output_sound_1000_rows_hsv_mapping.wav"
audio.export(output_filename, format="wav")
print(f"Generated {output_filename}")

Generated output_sound_1000_rows_hsv_mapping.wav
